<a href="https://colab.research.google.com/github/jyotidabass/NLP-Projects/blob/main/How_to_Run_DeepSeek_Locally.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/deepseek-ai/Janus.git

Cloning into 'Janus'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 121 (delta 51), reused 36 (delta 36), pack-reused 47 (from 2)
Receiving objects: 100% (121/121), 7.19 MiB | 13.29 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
%cd Janus
!pip install -e .
!pip install flash-attn

/content/Janus
Obtaining file:///content/Janus
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor
from janus.utils.io import load_pil_images

# Define model path
model_path = "deepseek-ai/Janus-Pro-1B"

# Load processor and tokenizer
vl_chat_processor = VLChatProcessor.from_pretrained(model_path)
tokenizer = vl_chat_processor.tokenizer

# Load model with remote code enabled
vl_gpt = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

# Move model to GPU
vl_gpt = vl_gpt.to(torch.bfloat16).cuda().eval()


Some kwargs in processor config are unused and will not have any effect: mask_prompt, ignore_id, num_image_tokens, sft_format, add_special_token, image_tag. 


In [ ]:
import re
def get_company_name(image_path):
  image_path = image_path
  question = """Guess the name of the clinic given the logo of the image
                The output should only contain the name and no need anything extra
                The output should have only company name without any extra characters or thoughts from assistant
                Enclose the clinic name in double quotes"""

  conversation = [
      {"role": "<|User|>", "content": f"<image_placeholder>\n{question}", "images": [image_path]},
      {"role": "<|Assistant|>", "content": ""}
  ]
  # Load image
  pil_images = load_pil_images(conversation)

  # Prepare inputs for the model
  prepare_inputs = vl_chat_processor(conversations=conversation, images=pil_images, force_batchify=True).to(vl_gpt.device)
  inputs_embeds = vl_gpt.prepare_inputs_embeds(**prepare_inputs)

  # Generate response
  outputs = vl_gpt.language_model.generate(
      inputs_embeds=inputs_embeds,
      attention_mask=prepare_inputs.attention_mask,
      pad_token_id=tokenizer.eos_token_id,
      bos_token_id=tokenizer.bos_token_id,
      eos_token_id=tokenizer.eos_token_id,
      max_new_tokens=512,
      do_sample=False,
      use_cache=True,
  )

  # Decode and print response
  answer = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
  try:
    result = re.findall(r'"(.*?)"', answer)
    return result[0]
  except:
    return answer

In [ ]:
!pip install faker

In [ ]:
from faker import Faker
import random

# Medical Bill

In [ ]:
def populate_records():
    faker_object = Faker()
    # Generate values for the placeholders
    patient_name = faker_object.name()
    date_of_visit = faker_object.date()
    doctor_name = "Dr."+faker_object.name()
    insurance_provider = faker_object.company()
    description = 'General consulation'

    # Generate granular details (medicines/services)
    granular_data = []
    items = ['Paracetamol','Azithromycine','Novamax']
    quantities = [str(random.randint(2,10)),
                  str(random.randint(2,10)),
                  str(random.randint(2,10))]
    rates = [str(random.randint(8,30)),
             str(random.randint(5,30)),
             str(random.randint(7,30))]
    for i in range(3):  # Generate 3 items
        item = items[i]
        quantity = quantities[i]
        rate = rates[i]
        total = float(quantity) * float(rate.replace('$', ''))  # Calculate total
        granular_data.append({
            'Item': item,
            'Quantity': quantity,
            'Rate': f"${rate}",
            'Total': f"${total:.2f}"
        })

    # Calculate the overall total amount
    total_amount = sum(float(item['Total'].replace('$', '')) for item in granular_data)
    return patient_name, date_of_visit, doctor_name, insurance_provider, \
            description, granular_data, total_amount

In [ ]:
drive_link = '/content/drive/MyDrive/omdena_legal'
import os
random.choice(os.listdir(drive_link))

'Generated_Medical_Bill'

'733-570-0937x341'

In [ ]:
from docx import Document
import os
import random
from docx.shared import Inches
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

def document_template(num_docs):
    faker_object = Faker()
    # Create a new Word document
    for i in range(num_docs):
      doc = Document()
      folder_choice = random.choice(os.listdir(f"{drive_link}/Medical"))
      logo_choice = random.choice(os.listdir(f"{drive_link}/Medical/{folder_choice}"))
      logo_path = f"{drive_link}/Medical/{folder_choice}/{logo_choice}"
      doc.add_picture(logo_path, width=Inches(1.5))  # Adjust width as needed
      doc.add_paragraph('\n')  # Add space after the logo

      # Add a header with clinic/hospital name
      company_name = get_company_name(logo_path)
      doc.add_heading(company_name, 0)
      faker_object = Faker()
      doc.add_paragraph(faker_object.address())
      doc.add_paragraph(f'Phone: {faker_object.phone_number()} | Email: info@{company_name.replace(" ", "")}.com')

      # Add a title for the bill
      doc.add_heading('Medical Bill', level=1)

      # Add patient and billing details in a table
      table = doc.add_table(rows=6, cols=2)
      table.style = 'Table Grid'

      # Fill the table with placeholders
      rows = table.rows
      rows[0].cells[0].text = 'Patient Name:'
      rows[0].cells[1].text = '[Patient Name]'
      rows[1].cells[0].text = 'Date of Visit:'
      rows[1].cells[1].text = '[Date of Visit]'
      rows[2].cells[0].text = 'Doctor Name:'
      rows[2].cells[1].text = '[Doctor Name]'
      rows[3].cells[0].text = 'Insurance Provider:'
      rows[3].cells[1].text = '[Insurance Provider]'
      rows[4].cells[0].text = 'Total Amount:'
      rows[4].cells[1].text = '[Total Amount]'
      rows[5].cells[0].text = 'Description of Services:'
      rows[5].cells[1].text = '[Description]'

      # Add a table for granular details (medicines, services, etc.)
      doc.add_paragraph('\nItemized Bill:')
      granular_table = doc.add_table(rows=1, cols=4)
      granular_table.style = 'Table Grid'

      # Add headers for the granular table
      header_row = granular_table.rows[0].cells
      header_row[0].text = 'Item'
      header_row[1].text = 'Quantity'
      header_row[2].text = 'Rate (USD)'
      header_row[3].text = 'Total (USD)'

      # Add placeholders for granular details
      for _ in range(3):  # Add 3 rows for medicines/services
          row = granular_table.add_row().cells
          row[0].text = '[Item]'
          row[1].text = '[Quantity]'
          row[2].text = '[Rate]'
          row[3].text = '[Total]'

      # Add additional paragraphs
      doc.add_paragraph('\nPayment Terms:')
      doc.add_paragraph('Payment is due within 30 days of the invoice date. Late payments may incur a fee of 1.5% per month.')

      doc.add_paragraph('\nDisclaimer:')
      doc.add_paragraph('This bill is generated based on the services provided. Please contact us for any discrepancies.')

      doc.add_paragraph('\nFollow-Up Instructions:')
      doc.add_paragraph('Please schedule a follow-up appointment if required. Contact our reception for availability.')

      # Add a signature section
      doc.add_paragraph('\n\nAuthorized Signature:')

      folder_choice = random.choice(os.listdir(f"{drive_link}/signature/Dataset_Signature_Final/Dataset"))
      subfolder = random.choice(['forge','real'])
      sign_choice = random.choice(os.listdir(f"{drive_link}/signature/Dataset_Signature_Final/Dataset/{folder_choice}/{subfolder}"))
      signature_path = f"{drive_link}/signature/Dataset_Signature_Final/Dataset/{folder_choice}/{subfolder}/{sign_choice}"
      doc.add_picture(signature_path, width=Inches(1.5))  # Adjust width as needed
      doctor = "Dr."+faker_object.name()
      doc.add_paragraph(f'{doctor}\nChief Medical Officer\n{company_name}')
      #doc.save(f'medical_bill_template_{i}.docx')
      data = populate_records()
      populate_to_document(data,doc, i)

def populate_to_document(data, document, nth_doc):
    # Load the template document
    doc = document

    # Replace placeholders with generated values
    for paragraph in doc.paragraphs:
        if '[Patient Name]' in paragraph.text:
            paragraph.text = paragraph.text.replace('[Patient Name]', data[0])
        if '[Date of Visit]' in paragraph.text:
            paragraph.text = paragraph.text.replace('[Date of Visit]', data[1])
        if '[Doctor Name]' in paragraph.text:
            paragraph.text = paragraph.text.replace('[Doctor Name]', data[2])
        if '[Insurance Provider]' in paragraph.text:
            paragraph.text = paragraph.text.replace('[Insurance Provider]', data[3])
        if '[Total Amount]' in paragraph.text:
            paragraph.text = paragraph.text.replace('[Total Amount]', f"${data[6]:.2f}")
        if '[Description]' in paragraph.text:
            paragraph.text = paragraph.text.replace('[Description]', data[4])

    # Replace placeholders in tables
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                if '[Patient Name]' in cell.text:
                    cell.text = cell.text.replace('[Patient Name]', data[0])
                if '[Date of Visit]' in cell.text:
                    cell.text = cell.text.replace('[Date of Visit]', data[1])
                if '[Doctor Name]' in cell.text:
                    cell.text = cell.text.replace('[Doctor Name]', data[2])
                if '[Insurance Provider]' in cell.text:
                    cell.text = cell.text.replace('[Insurance Provider]', data[3])
                if '[Total Amount]' in cell.text:
                    cell.text = cell.text.replace('[Total Amount]', f"${data[6]:.2f}")
                if '[Description]' in cell.text:
                    cell.text = cell.text.replace('[Description]', data[4])

    # Replace granular table placeholders with generated data
    granular_table = doc.tables[1]  # Second table in the document
    for i, item in enumerate(data[5]):
        row = granular_table.rows[i + 1].cells  # Skip header row
        row[0].text = item['Item']
        row[1].text = item['Quantity']
        row[2].text = item['Rate']
        row[3].text = item['Total']

    # Save the populated document
    doc.save(rf'/content/drive/MyDrive/omdena_legal/Generated_Medical_Bill/generated_medical_bill_{nth_doc}.docx')


In [ ]:
document_template(num_docs=100)

# GDPR

In [ ]:
from docx import Document
from docx.shared import Inches

# Function to create GDPR policy document
def create_gdpr_policy(company_name, company_address, contact_email, logo_path, signature_path, output_file):
    # Create a new Word document
    doc = Document()

    # Add a title
    doc.add_heading(f'GDPR Policy Document for {company_name}', level=1)

    # Add the company logo
    if logo_path:
        doc.add_picture(logo_path, width=Inches(2.0))
        doc.add_paragraph(' ')  # Add some space after the logo

    # Section 1: Introduction
    doc.add_heading('1. Introduction', level=2)
    doc.add_paragraph(
        f"This document outlines the General Data Protection Regulation (GDPR) policy of {company_name}. "
        "It explains how we collect, process, and protect personal data in compliance with GDPR."
    )

    # Section 2: Scope
    doc.add_heading('2. Scope', level=2)
    doc.add_paragraph(
        "This policy applies to all employees, contractors, and third-party vendors who handle personal data "
        "on behalf of the company."
    )

    # Section 3: Personal Data Collected
    doc.add_heading('3. Personal Data Collected', level=2)
    doc.add_paragraph(
        "We may collect the following types of personal data (PII):"
    )
    pii_list = [
        "Full name",
        "Email address",
        "Phone number",
        "Home address",
        "IP address",
        "Bank account details (if applicable)"
    ]
    for item in pii_list:
        doc.add_paragraph(f"- {item}", style='List Bullet')

    # Section 4: Data Processing
    doc.add_heading('4. Data Processing', level=2)
    doc.add_paragraph(
        "Personal data is processed for the following purposes:"
    )
    processing_purposes = [
        "To fulfill contractual obligations",
        "To comply with legal requirements",
        "To improve our services",
        "For marketing purposes (with consent)"
    ]
    for purpose in processing_purposes:
        doc.add_paragraph(f"- {purpose}", style='List Bullet')

    # Section 5: Data Security
    doc.add_heading('5. Data Security', level=2)
    doc.add_paragraph(
        "We implement appropriate technical and organizational measures to ensure the security of personal data. "
        "This includes encryption, access controls, and regular audits."
    )

    # Section 6: Data Subject Rights
    doc.add_heading('6. Data Subject Rights', level=2)
    doc.add_paragraph(
        "Data subjects have the following rights under GDPR:"
    )
    subject_rights = [
        "Right to access personal data",
        "Right to rectification",
        "Right to erasure (right to be forgotten)",
        "Right to restrict processing",
        "Right to data portability",
        "Right to object"
    ]
    for right in subject_rights:
        doc.add_paragraph(f"- {right}", style='List Bullet')

    # Section 7: Contact Information
    doc.add_heading('7. Contact Information', level=2)
    doc.add_paragraph(
        f"For any questions or concerns regarding this policy, please contact us at:\n\n"
        f"Company Name: {company_name}\n"
        f"Address: {company_address}\n"
        f"Email: {contact_email}"
    )

    # Add a signature section
    doc.add_heading('Signature', level=2)
    doc.add_paragraph("I hereby confirm that I have read and understood the GDPR policy of the company.")
    if signature_path:
        doc.add_picture(signature_path, width=Inches(1.5))
    else:
        doc.add_paragraph("[Signature Placeholder]")

    doc.save(rf'/content/drive/MyDrive/omdena_legal/GDPR/{output_file}')

In [ ]:
faker_object = Faker()
for i in range(50):
  folder_choice = random.choice(os.listdir(f"{drive_link}/Medical"))
  logo_choice = random.choice(os.listdir(f"{drive_link}/Medical/{folder_choice}"))
  logo_path = f"{drive_link}/Medical/{folder_choice}/{logo_choice}"
  company_name = get_company_name(logo_path)
  company_address = faker_object.address()
  contact_email = f'info@{company_name.replace(" ", "")}.com'
  folder_choice = random.choice(os.listdir(f"{drive_link}/signature/Dataset_Signature_Final/Dataset"))
  subfolder = random.choice(['forge','real'])
  sign_choice = random.choice(os.listdir(f"{drive_link}/signature/Dataset_Signature_Final/Dataset/{folder_choice}/{subfolder}"))
  signature_path = f"{drive_link}/signature/Dataset_Signature_Final/Dataset/{folder_choice}/{subfolder}/{sign_choice}"
  create_gdpr_policy(company_name, company_address,
                     contact_email, logo_path,
                     signature_path, f"GDPR_generated_{i}.docx")

# Docs to pdf

In [3]:
from docx2pdf import convert

d:\Raghu Studies\omdena\CameroonFranceChapter_LegalComplianceAssistant\.src_omdena_legal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import json
with open(r"D:\Raghu Studies\omdena\CameroonFranceChapter_LegalComplianceAssistant\data\legaldoc_all_ann.json") as f:
  annotations = json.load(f)

In [8]:
for docname in annotations:
  doc_name_pdf = docname['documentName']
  doc_name = doc_name_pdf.split(".")[0]+".docx"
  convert(rf"D:\Raghu Studies\omdena\CameroonFranceChapter_LegalComplianceAssistant\data\divorce\{doc_name}", doc_name_pdf )



100%|██████████| 1/1 [00:01<00:00,  1.63s/it]


100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


100%|██████████| 1/1 [00:01<00:00,  1.98s/it]


100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


100%|██████████| 1/1 [00:01<00:00,  1.83s/it]


100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


100%|██████████| 1/1 [00:01<00:00,  1.76s/it]


100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


100%|██████████| 1/1 [00:01<00:00,  1.79s/it]


In [ ]:
import os
os.listdir()

  Obtaining dependency information for pdf2image from https://files.pythonhosted.org/packages/62/33/61766ae033518957f877ab246f87ca30a85b778ebaad65b7f74fa7e52988/pdf2image-1.17.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import os
pdf_list = os.listdir(r'D:\Raghu Studies\omdena\CameroonFranceChapter_LegalComplianceAssistant\notebook')
pdf_list = [pdfs for pdfs in pdf_list if(".pdf" in pdfs)]
pdf_list

['Detailed_Divorce_Agreement_0.pdf',
 'Detailed_Divorce_Agreement_1.pdf',
 'Detailed_Divorce_Agreement_10.pdf',
 'Detailed_Divorce_Agreement_2.pdf',
 'Detailed_Divorce_Agreement_3.pdf',
 'Detailed_Divorce_Agreement_4.pdf',
 'Detailed_Divorce_Agreement_5.pdf',
 'Detailed_Divorce_Agreement_7.pdf',
 'Detailed_Divorce_Agreement_8.pdf',
 'Detailed_Divorce_Agreement_9.pdf']

In [12]:
from pdf2image import convert_from_path
from PIL import Image

for pdfs in pdf_list:
    file_name_image = pdfs.split(".")[0]
    pages = convert_from_path(pdfs, 500, poppler_path=r'C:\Program Files\poppler-24.08.0\Library\bin')
    
    for count, page in enumerate(pages):
        resized_page = page.resize((762, 1000)) # to match the resolution what is present in funsd dataset
        
        resized_page.save(rf'D:\Raghu Studies\omdena\CameroonFranceChapter_LegalComplianceAssistant\notebook\images\{file_name_image}_pg{count}.jpg', 'JPEG')

In [ ]:
from pdf2image import convert_from_path
for pdfs in pdf_list:
    file_name_image = pdfs.split(".")[0]
    pages = convert_from_path(pdfs, 500, poppler_path=r'C:\Program Files\poppler-24.08.0\Library\bin')
    for count, page in enumerate(pages):
        page.save(rf'D:\Raghu Studies\omdena\CameroonFranceChapter_LegalComplianceAssistant\notebook\images\{file_name_image}_pg{count}.jpg', 'JPEG')

: 

In [ ]:
#inception
# admin
# admin1234
# java -jar inception-app-webapp-35.2-standalone.jar

'Detailed_Divorce_Agreement_3.pdf'

# Convert Docx to PDF

In [3]:
import os

In [4]:
PATH = r"D:\Raghu Studies\omdena\CameroonFranceChapter_LegalComplianceAssistant\data\GDPR"

In [ ]:
files = os.listdir(PATH)
import os
import win32com.client
files = os.listdir(PATH)

def convert_docx_to_pdf(input_path, output_path=None):
    print(input_path)
    word = win32com.client.Dispatch("Word.Application")
    word.Visible = False  # Run Word in the background

    try:
        doc = word.Documents.Open(os.path.abspath(input_path))

        doc.SaveAs(os.path.abspath(output_path), FileFormat=17)  # 17 is the PDF format
        doc.Close()
        print(f"Converted {input_path} to {output_path}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        word.Quit()


for in_file in files:
    doc_path = PATH + rf"\{in_file}"
    os.makedirs(PATH + rf"\pdf", exist_ok=True)
    output_path = PATH + rf"\pdf\{in_file.split('.docx')[0]}.pdf"
    convert_docx_to_pdf(doc_path, output_path)


# Convert PDF to images

In [2]:
from pdf2image import convert_from_path
import os

In [7]:
def convert_pdf_to_images(path):
    path_files = os.listdir(path)[:10] # take only 10 files
    output_path = r"D:/Raghu Studies/omdena/CameroonFranceChapter_LegalComplianceAssistant/data/yolo_training_images"
    pdf_list = [path + rf"/{files}" for files in path_files]
    for pdfs in pdf_list:
        file_name_image = pdfs.split(r"/")[-1].split(".")[0]
        pages = convert_from_path(pdfs, 500)

        for count, page in enumerate(pages):

            page.save(rf'{output_path}/{file_name_image}_pg{count}.jpg', 'JPEG')
    return True

In [9]:
from tqdm import tqdm
paths = [r'D:/Raghu Studies/omdena/CameroonFranceChapter_LegalComplianceAssistant/data/Dissolution_Petitions/pdf',
         r'D:/Raghu Studies/omdena/CameroonFranceChapter_LegalComplianceAssistant/data/Financial_bills',
         r'D:/Raghu Studies/omdena/CameroonFranceChapter_LegalComplianceAssistant/data/Generated_Medical_Bill/pdf',
         r'D:/Raghu Studies/omdena/CameroonFranceChapter_LegalComplianceAssistant/data/Lease_Agreements/pdf',
         r'D:/Raghu Studies/omdena/CameroonFranceChapter_LegalComplianceAssistant/data/NDA_Documents/pdf',
         r'D:/Raghu Studies/omdena/CameroonFranceChapter_LegalComplianceAssistant/data/Partnership_Agreements/pdf']

for path in tqdm(paths):
    convert_pdf_to_images(path)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [01:33<00:00, 15.63s/it]
